# Wholesale Customer Segmentation Analysis

This notebook performs customer segmentation on the UCI Wholesale Customers dataset using k-Means clustering and Agglomerative Clustering for comparison.

## Table of Contents
1. [Data Loading and Exploration](#data-loading)
2. [Exploratory Data Analysis](#eda)
3. [Data Preprocessing](#preprocessing)
4. [Determining Optimal Number of Clusters](#optimal-k)
5. [k-Means Clustering Implementation](#kmeans)
6. [Agglomerative Clustering Comparison](#agglomerative)
7. [Principal Component Analysis](#pca)
8. [Cluster Profiling and Business Personas](#personas)
9. [Model Persistence](#persistence)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import scipy.cluster.hierarchy as sch
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# Configure plotting settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

: 

<a id='data-loading'></a>
## 1. Data Loading and Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('../Wholesale_customers_data.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Dataset information
print("Dataset Info:")
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
df.isnull().sum()

<a id='eda'></a>
## 2. Exploratory Data Analysis

In [ ]:
# Distribution of spending across categories
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

columns = ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen']

for i, col in enumerate(columns):
    axes[i].hist(df[col], bins=30, color=f'C{i}', alpha=0.7)
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df[columns].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Heatmap of Spending Categories')
plt.show()

In [ ]:
# Boxplots to detect outliers
plt.figure(figsize=(15, 8))
df_melted = pd.melt(df, value_vars=columns)
sns.boxplot(x='variable', y='value', data=df_melted)
plt.title('Boxplots for Outlier Detection')
plt.xticks(rotation=45)
plt.yscale('log')  # Log scale to better visualize spread
plt.ylabel('Spending (Log Scale)')
plt.show()

<a id='preprocessing'></a>
## 3. Data Preprocessing

In [ ]:
# Prepare data for clustering (exclude Channel and Region)
X = df[columns]
print("Features for clustering:")
X.head()

In [ ]:
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Scaled data shape:", X_scaled.shape)
print("Mean of scaled data (should be ~0):", np.mean(X_scaled, axis=0))
print("Std of scaled data (should be ~1):", np.std(X_scaled, axis=0))

<a id='optimal-k'></a>
## 4. Determining Optimal Number of Clusters

In [ ]:
# Elbow Method
inertias = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, marker='o', linestyle='-', color='b')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

In [ ]:
# Silhouette Analysis
silhouette_scores = []
K_range_sil = range(2, 11)  # Silhouette score requires at least 2 clusters

for k in K_range_sil:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(K_range_sil, silhouette_scores, marker='o', linestyle='-', color='g')
plt.title('Silhouette Score Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Average Silhouette Score')
plt.grid(True)
plt.show()

# Print silhouette scores
for i, score in enumerate(silhouette_scores):
    print(f"k={i+2}: Silhouette Score = {score:.4f}")

In [ ]:
# Calinski-Harabasz Index
ch_scores = []

for k in K_range_sil:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    ch_score = calinski_harabasz_score(X_scaled, cluster_labels)
    ch_scores.append(ch_score)

# Plot Calinski-Harabasz scores
plt.figure(figsize=(10, 6))
plt.plot(K_range_sil, ch_scores, marker='o', linestyle='-', color='r')
plt.title('Calinski-Harabasz Index for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Calinski-Harabasz Score')
plt.grid(True)
plt.show()

# Print CH scores
for i, score in enumerate(ch_scores):
    print(f"k={i+2}: Calinski-Harabasz Score = {score:.2f}")

Based on the Elbow Method, Silhouette Score, and Calinski-Harabasz Index, let's select k=3 as the optimal number of clusters.

<a id='kmeans'></a>
## 5. k-Means Clustering Implementation

In [ ]:
# Apply k-Means clustering with k=3
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Add cluster labels to the original dataframe
df['Cluster'] = cluster_labels

print("k-Means clustering completed.")
print(f"Number of clusters: {optimal_k}")
print(f"Silhouette Score: {silhouette_score(X_scaled, cluster_labels):.4f}")
print(f"Calinski-Harabasz Score: {calinski_harabasz_score(X_scaled, cluster_labels):.2f}")

In [ ]:
# Count of customers in each cluster
cluster_counts = df['Cluster'].value_counts().sort_index()
print("Customer distribution across clusters:")
print(cluster_counts)

# Visualization
plt.figure(figsize=(8, 5))
sns.barplot(x=cluster_counts.index, y=cluster_counts.values, palette='viridis')
plt.title('Customer Distribution Across Clusters')
plt.xlabel('Cluster')
plt.ylabel('Number of Customers')
plt.show()

<a id='agglomerative'></a>
## 6. Agglomerative Clustering Comparison

In [ ]:
# Apply Agglomerative Clustering with k=3
agg_clustering = AgglomerativeClustering(n_clusters=optimal_k)
agg_labels = agg_clustering.fit_predict(X_scaled)

# Add Agglomerative cluster labels to the dataframe
df['Agg_Cluster'] = agg_labels

print("Agglomerative Clustering completed.")
print(f"Silhouette Score: {silhouette_score(X_scaled, agg_labels):.4f}")
print(f"Calinski-Harabasz Score: {calinski_harabasz_score(X_scaled, agg_labels):.2f}")

In [ ]:
# Compare cluster distributions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# k-Means distribution
kmeans_counts = df['Cluster'].value_counts().sort_index()
axes[0].bar(kmeans_counts.index, kmeans_counts.values, color='skyblue')
axes[0].set_title('k-Means Clustering Distribution')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Number of Customers')

# Agglomerative distribution
agg_counts = df['Agg_Cluster'].value_counts().sort_index()
axes[1].bar(agg_counts.index, agg_counts.values, color='lightcoral')
axes[1].set_title('Agglomerative Clustering Distribution')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

In [ ]:
# Compare silhouette scores
kmeans_silhouette = silhouette_score(X_scaled, cluster_labels)
agg_silhouette = silhouette_score(X_scaled, agg_labels)

comparison_df = pd.DataFrame({
    'Algorithm': ['k-Means', 'Agglomerative'],
    'Silhouette Score': [kmeans_silhouette, agg_silhouette]
})

print("Algorithm Comparison:")
print(comparison_df)

# Visualization
plt.figure(figsize=(8, 5))
sns.barplot(x='Algorithm', y='Silhouette Score', data=comparison_df, palette='Set2')
plt.title('Silhouette Score Comparison')
plt.ylabel('Silhouette Score')
plt.show()

We'll proceed with k-Means clustering as it typically provides better results for this type of data.

<a id='pca'></a>
## 7. Principal Component Analysis

In [ ]:
# Apply PCA to reduce dimensions to 2 components
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print("PCA completed.")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")

In [ ]:
# Create a DataFrame with PCA results
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Cluster'] = df['Cluster']

# Plot PCA results
plt.figure(figsize=(12, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['Cluster'], cmap='viridis', alpha=0.7)
plt.title('PCA - 2D Visualization of Customer Segments')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance explained)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance explained)')
plt.colorbar(scatter)
plt.grid(True)
plt.show()

In [ ]:
# Feature importance in PCA components
components_df = pd.DataFrame(
    pca.components_, 
    columns=columns, 
    index=['PC1', 'PC2']
)

plt.figure(figsize=(10, 6))
sns.heatmap(components_df.T, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('PCA Components - Feature Importance')
plt.ylabel('Original Features')
plt.xlabel('Principal Components')
plt.show()

<a id='personas'></a>
## 8. Cluster Profiling and Business Personas

In [ ]:
# Calculate mean spending per cluster
cluster_means = df.groupby('Cluster')[columns].mean()
print("Mean spending per cluster:")
cluster_means

In [ ]:
# Visualization of cluster means
cluster_means.plot(kind='bar', figsize=(12, 8))
plt.title('Average Spending by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Average Spending')
plt.legend(title='Product Category')
plt.xticks(rotation=0)
plt.yscale('log')
plt.grid(True, axis='y')
plt.show()

In [ ]:
# Identify dominant categories for each cluster
dominant_categories = {}
weakest_categories = {}

for cluster in cluster_means.index:
    cluster_data = cluster_means.loc[cluster]
    dominant_categories[cluster] = cluster_data.idxmax()
    weakest_categories[cluster] = cluster_data.idxmin()
    
print("Dominant and weakest categories by cluster:")
for cluster in cluster_means.index:
    print(f"Cluster {cluster}: Dominant = {dominant_categories[cluster]}, Weakest = {weakest_categories[cluster]}")

In [ ]:
# Generate business personas
personas = {}

for cluster in cluster_means.index:
    cluster_data = cluster_means.loc[cluster]
    dominant_category = dominant_categories[cluster]
    weakest_category = weakest_categories[cluster]
    
    # Create behavioral tags
    if dominant_category == 'Fresh':
        behavioral_tag = "Fresh Produce Specialists"
    elif dominant_category == 'Grocery':
        behavioral_tag = "Grocery-Dominant Retailers"
    elif dominant_category == 'Milk':
        behavioral_tag = "Dairy Product Wholesalers"
    elif dominant_category == 'Frozen':
        behavioral_tag = "Frozen Food Distributors"
    elif dominant_category == 'Detergents_Paper':
        behavioral_tag = "Non-Food Essentials Suppliers"
    else:  # Delicassen
        behavioral_tag = "Specialty Food Providers"
    
    # Create marketing insights
    if cluster == 0:
        marketing_insight = "These customers are high-volume fresh produce buyers. Target with premium quality offerings and seasonal promotions."
        campaign_recommendation = "Promote organic and specialty fresh produce lines with volume discounts."
    elif cluster == 1:
        marketing_insight = "These customers focus heavily on grocery items. They represent stable, recurring revenue opportunities."
        campaign_recommendation = "Introduce loyalty programs and bundled grocery packages."
    else:  # cluster == 2
        marketing_insight = "These customers have diverse purchasing patterns with balanced spending across categories."
        campaign_recommendation = "Offer cross-category promotions and personalized recommendations."
    
    # Create persona summary
    persona_summary = f"{behavioral_tag} who primarily purchase {dominant_category.lower()} products. "
    persona_summary += f"They show minimal interest in {weakest_category.lower()} items. "
    persona_summary += f"{marketing_insight}"
    
    personas[cluster] = {
        'cluster_id': cluster,
        'size': int(cluster_counts[cluster]),
        'dominant_category': dominant_category,
        'weakest_category': weakest_category,
        'behavioral_tag': behavioral_tag,
        'mean_spending': cluster_data.to_dict(),
        'persona_summary': persona_summary,
        'marketing_insight': marketing_insight,
        'campaign_recommendation': campaign_recommendation
    }

# Display personas
for cluster_id, persona in personas.items():
    print(f"\n===== CLUSTER {cluster_id} PERSONA =====")
    print(f"Behavioral Tag: {persona['behavioral_tag']}")
    print(f"Dominant Category: {persona['dominant_category']}")
    print(f"Weakest Category: {persona['weakest_category']}")
    print(f"Cluster Size: {persona['size']} customers")
    print(f"Persona Summary: {persona['persona_summary']}")
    print(f"Campaign Recommendation: {persona['campaign_recommendation']}")

In [ ]:
# Save personas to JSON
with open('../ml/personas.json', 'w') as f:
    json.dump(personas, f, indent=2)

print("Personas saved to ../ml/personas.json")

In [ ]:
# Create markdown version of personas
with open('../ml/personas.md', 'w') as f:
    f.write("# Customer Segment Personas\n\n")
    for cluster_id, persona in personas.items():
        f.write(f"## Cluster {cluster_id}: {persona['behavioral_tag']}\n\n")
        f.write(f"**Dominant Category:** {persona['dominant_category']}  \n")
        f.write(f"**Weakest Category:** {persona['weakest_category']}  \n")
        f.write(f"**Cluster Size:** {persona['size']} customers  \n\n")
        f.write(f"**Persona Summary:** {persona['persona_summary']}  \n\n")
        f.write(f"**Marketing Recommendation:** {persona['campaign_recommendation']}  \n\n")
        f.write("---\n\n")

print("Personas saved to ../ml/personas.md")

<a id='persistence'></a>
## 9. Model Persistence

In [ ]:
# Save the trained models
import os

# Create ml directory if it doesn't exist
os.makedirs('../ml', exist_ok=True)

# Save k-Means model
with open('../ml/kmeans.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

# Save scaler
with open('../ml/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save PCA transformer
with open('../ml/pca.pkl', 'wb') as f:
    pickle.dump(pca, f)

print("Models saved successfully:")
print("- kmeans.pkl")
print("- scaler.pkl")
print("- pca.pkl")

In [ ]:
# Save processed data for backend use
df.to_csv('../ml/processed_data.csv', index=False)
pca_df.to_csv('../ml/pca_data.csv', index=False)

print("Processed data saved:")
print("- processed_data.csv")
print("- pca_data.csv")

In [ ]:
# Save elbow data for backend API
elbow_data = {
    'k_values': list(K_range),
    'inertias': inertias
}

with open('../ml/elbow_data.json', 'w') as f:
    json.dump(elbow_data, f)

print("Elbow data saved to ../ml/elbow_data.json")

## Conclusion

This notebook has successfully completed the customer segmentation analysis using k-Means clustering. We've:

1. Explored and preprocessed the wholesale customers dataset
2. Determined the optimal number of clusters using multiple methods
3. Implemented k-Means clustering and compared with Agglomerative Clustering
4. Applied PCA for dimensionality reduction and visualization
5. Profiled clusters and generated actionable business personas
6. Saved all models and data for use in the backend API

The next step is to implement the backend API that will serve these clustering results and personas to the frontend application.